In [22]:
from __future__ import annotations

import pandas as pd
from research.fractional_momentum.fractional_momentum.config.liquid_experiment_config import (
    ExperimentConfig,
)

In [23]:
SELECTED_FACTORS = ["low_risk", "momentum", "size", "quality", "value"]

jkp_factors = pd.read_csv(ExperimentConfig.PATH_INPUT / "jkp_factors.csv")
jkp_factors = jkp_factors[jkp_factors["name"].isin(SELECTED_FACTORS)]
jkp_factors["date"] = pd.to_datetime(jkp_factors["date"])

In [24]:
factors = jkp_factors.pivot_table(index="date", columns="name", values="ret")
factors

name,low_risk,momentum,quality,size,value
date,,,,,
1926-01-02,NaN,NaN,NaN,0.002354,NaN
1926-01-04,NaN,NaN,NaN,0.005459,NaN
1926-01-05,NaN,NaN,NaN,0.003777,NaN
1926-01-06,NaN,NaN,NaN,-0.000590,NaN
1926-01-07,NaN,NaN,NaN,0.002057,NaN
...,...,...,...,...,...
2024-12-24,-0.003400,0.001616,-0.000962,0.001315,-0.002292
2024-12-26,-0.003880,-0.000633,-0.002414,0.008669,-0.001183
2024-12-27,0.007210,-0.004166,0.000425,-0.003512,0.005444


In [25]:
data_df = pd.read_csv(ExperimentConfig.PATH_OUTPUT / "data_df.csv")
data_df["date"] = pd.to_datetime(data_df["date"])
data_df = data_df.set_index("date")

In [26]:
data_df[["spx"]]

,spx
date,
2004-03-19,-0.011173
2004-03-22,-0.012958
2004-03-23,-0.001324
2004-03-24,-0.002395
2004-03-25,0.016365
...,...
2024-12-24,0.011043
2024-12-26,-0.000406
2024-12-27,-0.011056


In [27]:
factors = factors.merge(data_df[["spx"]], left_index=True, right_index=True, how="left")

In [28]:
factors.to_csv(ExperimentConfig.PATH_OUTPUT / "factors.csv")